# GeekBrains University
## Methods of data collection

## Lesson 2

### Task 1 
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH.  
Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы).  
Получившийся список должен содержать в себе минимум:  
● Наименование вакансии.  
● Предлагаемую зарплату (отдельно минимальную и максимальную).  
● Ссылку на саму вакансию.  
● Сайт, откуда собрана вакансия.  
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение).  
Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [557]:
from bs4 import BeautifulSoup as bs
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import re

In [558]:
headers = {'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}

#### * Initially we are going to parse data 'as is'. We'll parse 'compensation' for min, max, and currency later.

### HeadHunter

In [559]:
occupation = 'санитар'

In [560]:
main_link = 'https://hh.ru'

In [561]:
response = requests.get(f'{main_link}/search/vacancy?area=1&st=searchVacancy&text={occupation}', headers=headers).text
soup = bs(response,'lxml')

#### Look for the number of pages in the footer:

In [562]:
pages = soup.find_all('a', {'class' : "bloko-button HH-Pager-Control"})
pprint(pages)

[<a class="bloko-button HH-Pager-Control" data-page="1" data-qa="pager-page" href="/search/vacancy?L_is_autosearch=false&amp;area=1&amp;clusters=true&amp;enable_snippets=true&amp;text=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80&amp;page=1" rel="nofollow">2</a>,
 <a class="bloko-button HH-Pager-Control" data-page="2" data-qa="pager-page" href="/search/vacancy?L_is_autosearch=false&amp;area=1&amp;clusters=true&amp;enable_snippets=true&amp;text=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80&amp;page=2" rel="nofollow">3</a>,
 <a class="bloko-button HH-Pager-Control" data-page="3" data-qa="pager-page" href="/search/vacancy?L_is_autosearch=false&amp;area=1&amp;clusters=true&amp;enable_snippets=true&amp;text=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80&amp;page=3" rel="nofollow">4</a>,
 <a class="bloko-button HH-Pager-Control" data-page="4" data-qa="pager-page" href="/search/vacancy?L_is_autosearch=false&amp;area=1&amp;clusters=true&amp;enable_snippets=true&amp;text=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0

In [563]:
pages = int(pages[-1].text)
print(f'Number of pages is {pages}')

Number of pages is 9


In [564]:
vacancy = []
for page in range(pages):
    link = f'{main_link}/search/vacancy?area=1&st=searchVacancy&text={occupation}&page={page}'
    response = requests.get(link, headers=headers).text
    soup = bs(response,'lxml')
    vacancy += (soup.find_all('div', {'data-qa':'vacancy-serp__vacancy'}))

In [565]:
print(len(vacancy))

437


In [566]:
pprint(vacancy[2:4])

[<div class="vacancy-serp-item" data-qa="vacancy-serp__vacancy"><script data-name="HH/VacancyResponseTrigger" data-params=""></script><div class="vacancy-serp-item__row vacancy-serp-item__row_header"><div class="vacancy-serp-item__info"><span class="bloko-section-header-3 bloko-section-header-3_lite"><span class="resume-search-item__name"><span class="g-user-content"><a class="bloko-link HH-LinkModifier" data-position="2" data-qa="vacancy-serp__vacancy-title" data-requestid="15865518062195a1af6fa7745219c173" data-totalvacancies="428" href="https://hh.ru/vacancy/36601291?query=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80" target="_blank">Санитарка/Санитар</a></span></span></span></div><div class="vacancy-serp-item__sidebar"><span class="bloko-section-header-3 bloko-section-header-3_lite" data-qa="vacancy-serp__vacancy-compensation">от 100 000 руб.</span></div></div><div class="vacancy-serp-item__meta-info"><a class="bloko-link bloko-link_secondary" data-qa="vacancy-serp__vacancy-employer"

In [567]:
base = []
i = 0
for v in vacancy:
    v_data = {}
    
    title = v.find('a', {'data-qa':'vacancy-serp__vacancy-title'})
    if not title:
        v_data['title'] = np.nan
    else:
        v_data['title'] = title.text
    
    link = v.find('a', {'data-qa':'vacancy-serp__vacancy-title'})
    if not link:
        v_data['link'] = np.nan
    else:
        v_data['link'] = link['href']
        
    employer = v.find('a', {'data-qa':'vacancy-serp__vacancy-employer'})
    if not employer:
        v_data['employer'] = np.nan
    else:
        v_data['employer'] = employer.text
    
    compensation = v.find('span', {'data-qa':'vacancy-serp__vacancy-compensation'})
    if not compensation:
        v_data['compensation'] = np.nan
    else:
        v_data['compensation'] = compensation.text
    
    v_data['source'] = main_link
    
    base.append(v_data)
    if i < 6:
        i += 1
        print(v_data)
        print('\n ------------------------------------------------------------- \n')

{'title': 'Санитар/санитарка', 'link': 'https://hh.ru/vacancy/36577813?query=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80', 'employer': ' Амориа', 'compensation': 'от 40\xa0000 руб.', 'source': 'https://hh.ru'}

 ------------------------------------------------------------- 

{'title': 'Санитарка/Санитар в аптеку (м. Раменки)', 'link': 'https://hh.ru/vacancy/36381593?query=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80', 'employer': 'АО Столичные аптеки', 'compensation': 'от 28\xa0000 руб.', 'source': 'https://hh.ru'}

 ------------------------------------------------------------- 

{'title': 'Санитарка/Санитар', 'link': 'https://hh.ru/vacancy/36601291?query=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80', 'employer': 'ООО Гарант-Мед', 'compensation': 'от 100\xa0000 руб.', 'source': 'https://hh.ru'}

 ------------------------------------------------------------- 

{'title': 'Санитар (ветклиника)', 'link': 'https://hh.ru/vacancy/35433244?query=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80', 'employe

In [568]:
len(base)

437

In [569]:
with open(f'HeadHunter.txt', 'w') as outfile:
    json.dump(base, outfile)

In [570]:
df_hh = pd.DataFrame(base)
df_hh.head()

,compensation,employer,link,source,title
0,от 40 000 руб.,Амориа,https://hh.ru/vacancy/36577813?query=%D1%81%D0...,https://hh.ru,Санитар/санитарка
1,от 28 000 руб.,АО Столичные аптеки,https://hh.ru/vacancy/36381593?query=%D1%81%D0...,https://hh.ru,Санитарка/Санитар в аптеку (м. Раменки)
2,от 100 000 руб.,ООО Гарант-Мед,https://hh.ru/vacancy/36601291?query=%D1%81%D0...,https://hh.ru,Санитарка/Санитар
3,NaN,"АО Август, Фирма",https://hh.ru/vacancy/35433244?query=%D1%81%D0...,https://hh.ru,Санитар (ветклиника)
4,34 000-34 000 руб.,ООО Нейро-Клиника,https://hh.ru/vacancy/36472038?query=%D1%81%D0...,https://hh.ru,Санитарка/санитар


In [571]:
df_hh.to_csv(f'HeadHunter.csv')

### Superjob

In [572]:
occupation = occupation #'курьер'

In [573]:
main_link = 'https://www.superjob.ru'

In [574]:
response = requests.get(f'{main_link}/vacancy/search/?keywords={occupation}', headers=headers).text
soup = bs(response,'lxml')

#### Look for the number of pages in the footer:

In [575]:
pages = soup.find('div', {'class' : "L1p51"}).find_all('span', {'class' : "_3IDf-"})
pprint(pages)

[<span class="_3IDf-">1</span>,
 <span class="_3IDf-">2</span>,
 <span class="_3IDf-">3</span>,
 <span class="_3IDf-">4</span>,
 <span class="_3IDf-">Дальше</span>]


In [576]:
pages = int(pages[-2].text)
print(f'Number of pages is {pages}')

Number of pages is 4


In [577]:
vacancy = []
for page in range(pages):
    link = f'{main_link}/vacancy/search/?keywords={occupation}&page={page}'
    response = requests.get(link, headers=headers).text
    soup = bs(response,'lxml')
    vacancy += (soup.find_all('div', {'class':'_3zucV f-test-vacancy-item _3j3cA RwN9e _3tNK- _1NStQ _1I1pc'}))

In [578]:
len(vacancy)

80

In [579]:
base = []
i = 0
for v in vacancy:
    v_data = {}
    
    title = v.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
    if not title:
        v_data['title'] = np.nan
    else:
        title = title.find('a')
        if not title:
            v_data['title'] = np.nan
        else:
            v_data['title'] = title.text
    
    link = v.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
    if not link:
        v_data['link'] = np.nan
    else:
        link = link.find('a')
        if not link:
            v_data['link'] = np.nan
        else:
            v_data['link'] = main_link + link['href']
    
    employer = v.find('span', {'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _2VHxz _15msI'})
    if not employer:
        v_data['employer'] = np.nan
    else:
        employer = employer.find('a')
        if not employer:
            v_data['employer'] = np.nan
        else:
            v_data['employer'] = employer.text
    
    compensation = v.find('span', {'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})
    if not compensation:
        v_data['compensation'] = np.nan
    else:
        v_data['compensation'] = compensation.text
        
    v_data['source'] = main_link
        
    base.append(v_data)
    if i < 6:
        i += 1
        print(v_data)
        print('\n ------------------------------------------------------------- \n')

{'title': 'Санитар / Санитарка (в лабораторию)', 'link': 'https://www.superjob.ru/vakansii/sanitar-32053850.html', 'employer': 'Независимая лаборатория ИНВИТРО', 'compensation': 'от\xa032\xa0000\xa0руб.', 'source': 'https://www.superjob.ru'}

 ------------------------------------------------------------- 

{'title': 'Санитарка / Санитар', 'link': 'https://www.superjob.ru/vakansii/sanitarka-32650853.html', 'employer': 'ГОРОДСКАЯ СЛУЖБА СОЦИАЛЬНОЙ ПОМОЩИ', 'compensation': '27\xa0000\xa0—\xa030\xa0000\xa0руб.', 'source': 'https://www.superjob.ru'}

 ------------------------------------------------------------- 

{'title': 'Санитарка / Санитар', 'link': 'https://www.superjob.ru/vakansii/sanitarka-33723759.html', 'employer': 'ЕВРОПЕЙСКИЙ МЕДИЦИНСКИЙ ЦЕНТР', 'compensation': '35\xa0000\xa0руб.', 'source': 'https://www.superjob.ru'}

 ------------------------------------------------------------- 

{'title': 'Санитар (в новую больницу поселения Вороновское)', 'link': 'https://www.superjob.ru/va

In [580]:
len(base)

80

In [581]:
with open(f'SuperJob.txt', 'w') as outfile:
    json.dump(base, outfile)

In [582]:
df_sj = pd.DataFrame(base)
df_sj.head()

,compensation,employer,link,source,title
0,от 32 000 руб.,Независимая лаборатория ИНВИТРО,https://www.superjob.ru/vakansii/sanitar-32053...,https://www.superjob.ru,Санитар / Санитарка (в лабораторию)
1,27 000 — 30 000 руб.,ГОРОДСКАЯ СЛУЖБА СОЦИАЛЬНОЙ ПОМОЩИ,https://www.superjob.ru/vakansii/sanitarka-326...,https://www.superjob.ru,Санитарка / Санитар
2,35 000 руб.,ЕВРОПЕЙСКИЙ МЕДИЦИНСКИЙ ЦЕНТР,https://www.superjob.ru/vakansii/sanitarka-337...,https://www.superjob.ru,Санитарка / Санитар
3,до 130 000 руб.,Департамент здравоохранения города Москвы,https://www.superjob.ru/vakansii/sanitar-33708...,https://www.superjob.ru,Санитар (в новую больницу поселения Вороновское)
4,25 000 — 30 000 руб.,"Медицинский центр ""КОРСАКОВ""",https://www.superjob.ru/vakansii/sanitar-po-na...,https://www.superjob.ru,Санитар по надзору и сопровождению пациентов в...


In [583]:
df_sj.to_csv(f'Superjob.csv', encoding= 'utf-8')

### Data concatination

In [584]:
file = f'HeadHunter.csv'
df_hh = pd.read_csv(file, header= 0, encoding= 'utf-8', index_col= 0)
file = f'Superjob.csv'
df_sj = pd.read_csv(file, header= 0, encoding= 'utf-8', index_col= 0)

In [585]:
df_hh.head()

,compensation,employer,link,source,title
0,от 40 000 руб.,Амориа,https://hh.ru/vacancy/36577813?query=%D1%81%D0...,https://hh.ru,Санитар/санитарка
1,от 28 000 руб.,АО Столичные аптеки,https://hh.ru/vacancy/36381593?query=%D1%81%D0...,https://hh.ru,Санитарка/Санитар в аптеку (м. Раменки)
2,от 100 000 руб.,ООО Гарант-Мед,https://hh.ru/vacancy/36601291?query=%D1%81%D0...,https://hh.ru,Санитарка/Санитар
3,NaN,"АО Август, Фирма",https://hh.ru/vacancy/35433244?query=%D1%81%D0...,https://hh.ru,Санитар (ветклиника)
4,34 000-34 000 руб.,ООО Нейро-Клиника,https://hh.ru/vacancy/36472038?query=%D1%81%D0...,https://hh.ru,Санитарка/санитар


In [586]:
df_sj.tail()

,compensation,employer,link,source,title
75,По договорённости,Всероссийский научно-исследовательский институ...,https://www.superjob.ru/vakansii/starshij-mast...,https://www.superjob.ru,Старший мастер (отдел главного механика-энерге...
76,от 80 000 руб.,Всероссийский научно-исследовательский институ...,https://www.superjob.ru/vakansii/inzhener-ovik...,https://www.superjob.ru,Инженер ОВиК
77,35 000 — 40 000 руб.,"ТЦСО ""Щукино""",https://www.superjob.ru/vakansii/uborschik-slu...,https://www.superjob.ru,Уборщик служебных помещений
78,По договорённости,PepsiCo,https://www.superjob.ru/vakansii/operator-nala...,https://www.superjob.ru,Оператор-наладчик линии в производстве пищевой...
79,до 30 000 руб.,"Группа ""Гута""",https://www.superjob.ru/vakansii/inzhener-po-o...,https://www.superjob.ru,Инженер по охране труда и пожарной безопасност...


In [587]:
df = pd.concat([df_hh, df_sj], axis= 0, ignore_index= True, sort= False)
df.head()

,compensation,employer,link,source,title
0,от 40 000 руб.,Амориа,https://hh.ru/vacancy/36577813?query=%D1%81%D0...,https://hh.ru,Санитар/санитарка
1,от 28 000 руб.,АО Столичные аптеки,https://hh.ru/vacancy/36381593?query=%D1%81%D0...,https://hh.ru,Санитарка/Санитар в аптеку (м. Раменки)
2,от 100 000 руб.,ООО Гарант-Мед,https://hh.ru/vacancy/36601291?query=%D1%81%D0...,https://hh.ru,Санитарка/Санитар
3,NaN,"АО Август, Фирма",https://hh.ru/vacancy/35433244?query=%D1%81%D0...,https://hh.ru,Санитар (ветклиника)
4,34 000-34 000 руб.,ООО Нейро-Клиника,https://hh.ru/vacancy/36472038?query=%D1%81%D0...,https://hh.ru,Санитарка/санитар


In [588]:
df.tail()

,compensation,employer,link,source,title
512,По договорённости,Всероссийский научно-исследовательский институ...,https://www.superjob.ru/vakansii/starshij-mast...,https://www.superjob.ru,Старший мастер (отдел главного механика-энерге...
513,от 80 000 руб.,Всероссийский научно-исследовательский институ...,https://www.superjob.ru/vakansii/inzhener-ovik...,https://www.superjob.ru,Инженер ОВиК
514,35 000 — 40 000 руб.,"ТЦСО ""Щукино""",https://www.superjob.ru/vakansii/uborschik-slu...,https://www.superjob.ru,Уборщик служебных помещений
515,По договорённости,PepsiCo,https://www.superjob.ru/vakansii/operator-nala...,https://www.superjob.ru,Оператор-наладчик линии в производстве пищевой...
516,до 30 000 руб.,"Группа ""Гута""",https://www.superjob.ru/vakansii/inzhener-po-o...,https://www.superjob.ru,Инженер по охране труда и пожарной безопасност...


In [589]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 5 columns):
compensation    406 non-null object
employer        508 non-null object
link            517 non-null object
source          517 non-null object
title           517 non-null object
dtypes: object(5)
memory usage: 20.3+ KB


In [590]:
df.isnull().sum()

compensation    111
employer          9
link              0
source            0
title             0
dtype: int64

### 'Compensation' parsing

Let's look at the enemy face to face

In [591]:
compensations = df.compensation.unique()
compensations

array(['от 40\xa0000 руб.', 'от 28\xa0000 руб.', 'от 100\xa0000 руб.',
       nan, '34\xa0000-34\xa0000 руб.', 'от 30\xa0000 руб.',
       '26\xa0400-29\xa0000 руб.', '60\xa0000-60\xa0000 руб.',
       '65\xa0000-65\xa0000 руб.', 'от 60\xa0000 руб.',
       'до 29\xa0000 руб.', '35\xa0000-35\xa0000 руб.',
       '40\xa0000-50\xa0000 руб.', 'до 40\xa0000 руб.',
       'до 13\xa0500 руб.', '40\xa0000-60\xa0000 руб.',
       'от 32\xa0000 руб.', '70\xa0000-90\xa0000 руб.',
       'от 20\xa0000 руб.', '27\xa0000-30\xa0000 руб.',
       'от 29\xa0000 руб.', 'от 65\xa0000 руб.', 'от 150\xa0000 руб.',
       'от 80\xa0000 руб.', '100\xa0000-100\xa0000 руб.',
       '70\xa0000-80\xa0000 руб.', '60\xa0000-80\xa0000 руб.',
       '30\xa0000-40\xa0000 руб.', '70\xa0000-70\xa0000 руб.',
       '40\xa0000-45\xa0000 руб.', 'от 18\xa0000 руб.',
       'от 50\xa0000 руб.', 'до 68\xa0000 руб.', 'от 70\xa0000 руб.',
       'от 45\xa0000 руб.', 'от 25\xa0000 руб.',
       '35\xa0000-42\xa0000 руб.', '40\

In [592]:
df['compensation_min'] = np.nan
df['compensation_max'] = np.nan
df['compensation_currency'] = np.nan

filter = ~df['compensation'].isnull()

for i, _ in df[filter].iterrows():
    
    compensation_tt = df.loc[i, 'compensation'].replace(u'\xa0', ' ')
    
    numbers = '[\d+\s]*\d+' # pattern for number values
    
    fr = re.search(f'^от {numbers}', compensation_tt) # 'от 30 000'
    if fr:
        df.loc[i, 'compensation_min'] = fr[0][3:].replace(' ', '')
        compensation_tt = compensation_tt.replace(fr[0], '')
        
    fr = re.search(f'^{numbers}[\s]*[\-—]+', compensation_tt) # '30 000-'
    if fr:
        df.loc[i, 'compensation_min'] = fr[0][:-1].replace(' ', '')
        compensation_tt = compensation_tt.replace(fr[0], '-')
    
    fr = re.search(f'^до {numbers}', compensation_tt) # 'до 30 000'
    if fr:
        df.loc[i, 'compensation_max'] = fr[0][3:].replace(' ', '')
        compensation_tt = compensation_tt.replace(fr[0], '')
        
        
    fr = re.search(f'^[\-—]+[\s]*{numbers}', compensation_tt) # '-30 000'
    if fr:
        df.loc[i, 'compensation_max'] = fr[0][1:].replace(' ', '')
        compensation_tt = compensation_tt.replace(fr[0], '')
        
    fr = re.search(f'^[\s]*{numbers}', compensation_tt) # '30 000'
    if fr:
        df.loc[i, 'compensation_max'] = fr[0].replace(' ', '')
        df.loc[i, 'compensation_min'] = fr[0].replace(' ', '')
        compensation_tt = compensation_tt.replace(fr[0], '')
    
    df.loc[i, 'compensation_currency'] = compensation_tt # the rest to the currency
    

In [593]:
df.head(10)

,compensation,employer,link,source,title,compensation_min,compensation_max,compensation_currency
0,от 40 000 руб.,Амориа,https://hh.ru/vacancy/36577813?query=%D1%81%D0...,https://hh.ru,Санитар/санитарка,40000,NaN,руб.
1,от 28 000 руб.,АО Столичные аптеки,https://hh.ru/vacancy/36381593?query=%D1%81%D0...,https://hh.ru,Санитарка/Санитар в аптеку (м. Раменки),28000,NaN,руб.
2,от 100 000 руб.,ООО Гарант-Мед,https://hh.ru/vacancy/36601291?query=%D1%81%D0...,https://hh.ru,Санитарка/Санитар,100000,NaN,руб.
3,NaN,"АО Август, Фирма",https://hh.ru/vacancy/35433244?query=%D1%81%D0...,https://hh.ru,Санитар (ветклиника),NaN,NaN,NaN
4,34 000-34 000 руб.,ООО Нейро-Клиника,https://hh.ru/vacancy/36472038?query=%D1%81%D0...,https://hh.ru,Санитарка/санитар,34000,34000,руб.
5,от 30 000 руб.,Лазерный Дом,https://hh.ru/vacancy/36373397?query=%D1%81%D0...,https://hh.ru,Санитарка/санитар в медицинскую клинику,30000,NaN,руб.
6,NaN,ООО Центр по развитию и применению традиционно...,https://hh.ru/vacancy/36231899?query=%D1%81%D0...,https://hh.ru,Санитарка/Санитар в стоматологию,NaN,NaN,NaN
7,26 400-29 000 руб.,СМ-Клиника,https://hh.ru/vacancy/35870660?query=%D1%81%D0...,https://hh.ru,Санитарка/Санитар,26400,29000,руб.
8,60 000-60 000 руб.,"Бородино, бизнес-отель",https://hh.ru/vacancy/36165731?query=%D1%81%D0...,https://hh.ru,Санитарный врач,60000,60000,руб.
9,65 000-65 000 руб.,ГК Медси. Клинические больницы,https://hh.ru/vacancy/36564994?query=%D1%81%D0...,https://hh.ru,Санитарка/Санитар,65000,65000,руб.


In [594]:
df['compensation_currency'].unique()

array([' руб.', nan, 'По договорённости'], dtype=object)

In [595]:
df.to_csv(f'hh_sj_completed.csv', encoding= 'utf-8')